In [1]:
import json
%reset


In [1]:
from pymongo import MongoClient
from lxml import etree
import dateparser
import datetime
import pandas as pd
import re


# TODO : Format date to interrop doc

def get_all_keys(doc, main_key=None, separator=".", key_list = [], first_call=True):
    '''
    Get all key and sub key of a document, sub key are constructed with a separator defined as a parameter.
    :param doc:
    :param main_key:
    :param separator:
    :param first_call
    :return:
    '''
    if isinstance(doc, type({})):

        for key in  doc.keys():
            if main_key is None :
                key_list.append(key)
                get_all_keys(doc[key],main_key=key, separator=separator, key_list=key_list, first_call=False)
            else :
                key_list.append(main_key+separator+key)
                get_all_keys(doc[key],main_key=main_key+separator+key, separator=separator, key_list=key_list, first_call=False)
    if isinstance(doc, type([])):
        for obj in doc:
            if isinstance(obj, type({})):
                for key in  obj.keys():
                    if main_key is None :
                        key_list.append(key)
                        get_all_keys(obj[key],main_key=key, separator=separator, key_list=key_list, first_call=False)
                    else :
                        key_list.append(main_key+separator+key)
                        get_all_keys(obj[key],main_key=main_key+separator+key, separator=separator, key_list=key_list, first_call=False)

    return key_list


def remove_id_in_json(f_json):
    """
    As mongoDB id or _id is reserved keyword, this function modify any field that is "id" or "_id" to add "doc" before.
    :param f_json: dict containing a parsed json
    :return: dict : f_json with a modified "id" or "_id" key if there was
    """
    key_list = ["id","_id"]
    if index_key_list:=[index for index, key_is_present in enumerate([key in f_json for key in key_list ]) if key_is_present]:
        for index in index_key_list :
            f_json["doc_"+ key_list[index]]= f_json.pop( key_list[index])
    return f_json

def format_date_json(doc):
    if type(doc) is dict:
        for key in doc.keys():
            if (type(doc[key]) is str) and (date := dateparser.parse(doc[key])) is not None:
                    doc[key]=date
            format_date_json(doc[key])
    if type(doc) is list:
        for object in doc :
            format_date_json(object)

    return doc


def read_preprocess_insert_in_mongodb_json(fp, mongodb_coll=None, fp_is_dict=False):
    """
    Read, remove any incompatible "id" key and insert the JSON in a collection in a mongodb database
    :param fp: str : file_path to a JSON to read
    :param mongodb_coll: MongoClient.database.collection : A collection in which insert files
    :return: None
    """
    if mongodb_coll is None:
        mongodb_coll = MongoClient("localhost:27017").no_model_name.interop_metadata
        print("coucou")
    try:
        # mongodb_coll.insert_one(format_date_json(remove_id_in_json(json.load(open(fp)))))
        # Error seens in data formatting
        if fp_is_dict:
            mongodb_coll.insert_one((remove_id_in_json(fp)))
        else:
            mongodb_coll.insert_one((remove_id_in_json(json.load(open(fp)))))
        # print(fp + " has been inserted successfully.")
    except Exception as exce :
        print("Insertion has not been successfully done. Logs : " + str(exce))

def dont_contains_dict(liste):
    for elem in liste :
        # print(elem)
        if type(elem) is dict:
            return False
    return True


# Transform a date to standard format
def format_date(date):
    # Transform a string date into a standard format by trying each
    # date format. If you want to add a format, add a try/except in the
    # last except
    # date : str : the date to transform
    # return : m : timedata : format is YYYY-MM-DD HH:MM:SS
    date_str = date
    #
    date_str = date_str.replace("st","").replace("th","")\
        .replace("nd","").replace("rd","").replace(" Augu "," Aug ")
    m = None
    sep_list = [".","/","-","_"," ",":"]
    for date_sep in sep_list:
        try:
            m = datetime.datetime.strptime(date_str, "%d"+date_sep+"%B"+date_sep+"%Y")
            break
        except ValueError:
            try:
                m = datetime.datetime.strptime(date_str, "%d"+date_sep+"%b"+date_sep+"%Y")
                break
            except ValueError:
                try:
                    m = datetime.datetime.strptime(date_str, "%Y"+date_sep+"%m"+date_sep+"%d")
                    break
                except ValueError:
                    try :
                        m = datetime.datetime.strptime(date_str,
                                                   "%d"+date_sep+"%m"+date_sep+"%Y")
                        break
                    except ValueError:
                        for hour_sep in sep_list:
                            try:
                                m = datetime.datetime\
                                    .strptime(date_str,"%d"+date_sep+"%m"+date_sep+"%Y %H"+hour_sep+"%M"+hour_sep+"%S")
                                break
                            except ValueError:
                                try:
                                    m = datetime.datetime\
                                        .strptime(date_str, "%Y"+date_sep+"%m"+date_sep+"%d %H"+hour_sep+"%M"+hour_sep+"%S")
                                    break
                                except ValueError:
                                    # HERE ADD A FORMAT TO CHECK
                                    # print("Format not recognised. \nConsider "
                                    #       "adding a date format "
                                    #       "in the function \"format_date\".")
                                    pass

    return m

def XML_to_dict(xml):
    xml_tag = re.sub("{.*}","",xml.tag)
    res_dict ={xml_tag:{}}
    path = [xml_tag]
    def tree_walk(xml, path):
        aux_path = path
        for child in xml.getchildren():
            child_tag=re.sub("{.*}","",child.tag)
            aux = res_dict
            for i in path:
                aux = aux[i]
            if child_tag in aux:
                for attr in child.attrib:
                    if attr in aux[child_tag]:

                        if isinstance(aux[child_tag][attr], list):
                            aux[child_tag][attr].append(child.attrib[attr])
                        else :
                            aux[child_tag][attr] = [aux[child_tag][attr]] + [child.attrib[attr]]
                    else :
                        aux[child_tag][attr] = child.attrib[attr]
                if isinstance(child.text,str):
                    if child.text.strip():
                        if "@value" in aux[child_tag]:
                            if isinstance(aux[child_tag]["@value"],list):

                                        aux[child_tag]["@value"].append(child.text)
                            else:
                                aux[child_tag]["@value"]=[aux[child_tag]["@value"]]+[child.text]
                        else:
                            aux[child_tag]["@value"]=child.text
            else:
                aux[child_tag]={}
                for attr in child.attrib:
                    aux[child_tag][attr]=child.attrib[attr]
                if isinstance(child.text,str):
                    if child.text.strip():
                        aux[child_tag]["@value"]=child.text
            tree_walk(child, aux_path+[child_tag])
    for child in xml.getchildren():
        child_tag=re.sub("{.*}","",child.tag)
        if child_tag in res_dict[xml_tag]:
            for attr in child.attrib:
                if attr in res_dict[xml_tag][child_tag]:
                    if isinstance(res_dict[xml_tag][child_tag][attr], list):
                        res_dict[xml_tag][child_tag][attr].append(child.attrib[attr])
                    else:
                        res_dict[xml_tag][child_tag][attr] = [res_dict[xml_tag][child_tag][attr]] + [child.attrib[attr]]
                else:
                    res_dict[xml_tag][child_tag][attr]=child.attrib[attr]
            if isinstance(child.text,str):
                if child.text.strip():
                    if "@value" in res_dict[xml_tag][child_tag]:
                        if isinstance(res_dict[xml_tag][child_tag]["@value"],list):
                                    res_dict[xml_tag][child_tag]["@value"].append(child.text)
                        else:
                            res_dict[xml_tag][child_tag]["@value"]=[res_dict[xml_tag][child_tag]["@value"]]+[child.text]
                    else:
                        res_dict[xml_tag][child_tag]["@value"]=child.text
        else:
            res_dict[xml_tag][child_tag]={}
            for attr in child.attrib:
                res_dict[xml_tag][child_tag][attr]=child.attrib[attr]
            if isinstance(child.text,str):
                if child.text.strip():
                    if "@value" in res_dict[xml_tag][child_tag]:
                        if isinstance(res_dict[xml_tag][child_tag]["@value"],list):
                            res_dict[xml_tag][child_tag]["@value"].append(child.text.strip())
                        else:
                            res_dict[xml_tag][child_tag]["@value"]=res_dict[xml_tag][child_tag]["@value"]+[child.text]

                    else:
                        res_dict[xml_tag][child_tag]["@value"]=child.text
        tree_walk(child, path+[child_tag])
    return res_dict

def from_keyset_to_csv(key_set, separator="."):
    color = {"0":"#F3722C","1":"#F8961E","2":"#F9C74F","3":"#90BE6D", "4":"#43AA8B","5":"#4D908E","6":"#577590","7":"#277DA1", "8":"#bdd5ea", "9":"#e3e2e3","10":"#ffffff", "11":"#ffffff"}
    csv_list = [("ROOT_NODE","",-1,"#F94144")]
    for key_concat in key_set:
        key_split = key_concat.split(separator)
        for index in range(len(key_split)):
            if index <9:
                if len(key_split)==1:
                    csv_list.append((separator.join(key_split[:index+1]),"ROOT_NODE", index, color[str(index)]))
                    break
                if index == len(key_split) -1:
                    break
                csv_list.append((separator.join(key_split[:index+2]),separator.join(key_split[:index+1]),index+1, color[str(index+1)]))

            else :
                if index == len(key_split) -1:
                    break
                csv_list.append((separator.join(key_split[:index+2]),separator.join(key_split[:index+1]),index+1,"#ffffff"))
    return (["key","mother_key", "level","color"],set(csv_list))

In [2]:
import os
import json


var = False


def check_value_in_dict(val,dict_to_check, res_list, path=None):
    for i in dict_to_check:
        if path is None:
            aux_path = i
        else :
            aux_path = path + "." + i
        # if isinstance(val, list):
        #     if dict_to_check[i] == val or dict_to_check[i] == val:
        #         pass
        if dict_to_check[i] == val :
            res_list.append(aux_path)
        if isinstance(dict_to_check[i],dict):
            check_value_in_dict(val,dict_to_check[i], res_list, aux_path)


def map_two_dict(dict_a,dict_b, res_tuple_list, path=None):
    for path_iterator_var in dict_a:
        if isinstance(dict_a[path_iterator_var],dict):
            if path is None :
                path_aux = path_iterator_var
            else :
                path_aux = path +"."+ path_iterator_var

            map_two_dict(dict_a[path_iterator_var], dict_b,res_tuple_list, path_aux)
        else :
            if isinstance(dict_a[path_iterator_var],list):
                contains_dict=False
                for index, element in enumerate(dict_a[path_iterator_var]):
                    if isinstance(element,dict):
                        contains_dict = True
                        if path is None :
                            path_aux = path_iterator_var
                        else :
                            path_aux = path +"."+str(index)+"."+ path_iterator_var
                        map_two_dict(element, dict_b,res_tuple_list, path_aux)
                if not contains_dict:
                    for index, element in enumerate(dict_a[path_iterator_var]):
                        list_aux=[]
                        check_value_in_dict(element,dict_b,list_aux)
                        if list_aux:
                            if path is None :
                                res_tuple_list.append((path_iterator_var,list_aux))
                            else :
                                res_tuple_list.append(( path +"."+str(index)+"."+ path_iterator_var,list_aux ))
            list_aux=[]
            check_value_in_dict(dict_a[path_iterator_var],dict_b,list_aux)
            if list_aux:
                if path is None :
                    res_tuple_list.append((path_iterator_var,list_aux))
                else :
                    res_tuple_list.append(( path +"."+ path_iterator_var,list_aux ))


res=[]
num_files = 0
for enumerator,folder in enumerate(list(os.walk("cda2r4/source"))):
    print(str(enumerator)+"/"+(str(len(list(os.walk("cda2r4/source"))))))
    if folder[2]:
        num_files+=len(folder[2])
        for index, file_name in enumerate(folder[2]):
            print(str(index)+"/"+str(len(folder[2])))
            var_input = folder[0] + "/" + file_name
            var_output = folder[0].replace("source","output")+ "/FHIR_" + file_name.replace(".xml",".json")
            input_xml = XML_to_dict(etree.parse(open(var_input), parser=etree.XMLParser(ns_clean=True, remove_comments=True, recover=True)).getroot())
            output_json = json.load(open(var_output))
            mapping_list = []
            map_two_dict(output_json,input_xml,mapping_list)
            for mapping in mapping_list :
                # Exclusion des multi mapping (1:n)
                if len(mapping[1])==1:
                    remove_number=[]
                    for tag in mapping[0].split("."):
                        if not tag.isdigit():
                            remove_number.append(tag)
                    # mapping[1][0] est une liste de taille 1, on prend juste le contenu
                    res.append((".".join(remove_number) , mapping[1][0]))

pd.DataFrame(set(res), columns=["key","match"]).set_index("key").to_csv("FHIR_to_C-CDA_._delimiter.csv")
print(str(num_files)+" files processed.")


0/21
1/21
0/9
1/9
2/9
3/9
4/9
5/9
6/9
7/9
8/9
2/21
0/3
1/3
2/3
3/21
4/21
0/2
1/2
5/21
0/5
1/5
2/5
3/5
4/5
6/21
0/13
1/13
2/13
3/13
4/13
5/13
6/13
7/13
8/13
9/13
10/13
11/13
12/13
7/21
0/1
8/21
0/2
1/2
9/21
0/1
10/21
0/5
1/5
2/5
3/5
4/5
11/21
12/21
13/21
0/6
1/6
2/6
3/6
4/6
5/6
14/21
0/1
15/21
0/12
1/12
2/12
3/12
4/12
5/12
6/12
7/12
8/12
9/12
10/12
11/12
16/21
0/2
1/2
17/21
0/683
1/683
2/683
3/683
4/683
5/683
6/683
7/683
8/683
9/683
10/683
11/683
12/683
13/683
14/683
15/683
16/683
17/683
18/683
19/683
20/683
21/683
22/683
23/683
24/683
25/683
26/683
27/683
28/683
29/683
30/683
31/683
32/683
33/683
34/683
35/683
36/683
37/683
38/683
39/683
40/683
41/683
42/683
43/683
44/683
45/683
46/683
47/683
48/683
49/683
50/683
51/683
52/683
53/683
54/683
55/683
56/683
57/683
58/683
59/683
60/683
61/683
62/683
63/683
64/683
65/683
66/683
67/683
68/683
69/683
70/683
71/683
72/683
73/683
74/683
75/683
76/683
77/683
78/683
79/683
80/683
81/683
82/683
83/683
84/683
85/683
86/683
87/683
88/683
89/683
90/6

In [119]:
import pprint

pprint.pprint(input_xml)

{'ClinicalDocument': {'author': {'assignedAuthor': {'addr': {'city': {'XML_transform_tag_content': 'Springfield'},
                                                             'country': {'XML_transform_tag_content': 'US'},
                                                             'postalCode': {'XML_transform_tag_content': '22150'},
                                                             'state': {'XML_transform_tag_content': 'VA'},
                                                             'streetAddressLine': {'XML_transform_tag_content': '1400 '
                                                                                                                'Main '
                                                                                                                'Street '
                                                                                                                'Ste '
                                                                         

In [3]:
import os
import time
from valentine import valentine_match
from valentine.algorithms import Coma
print("Starting matching..")
path_to_mapping = "../mappings/"
path_to_models = "../models/"
onlyfiles = [f for f in os.listdir(path_to_mapping) if
             os.path.isfile(os.path.join(path_to_mapping, f)) and f.endswith(".csv")]
for ground_truth_path in onlyfiles:
    print(ground_truth_path)
    parameters = ground_truth_path.split("_")
    modelA_name = parameters[0]
    modelB_name = parameters[2]
    delimiter = parameters[3]
    # print(parameters)
    # print(modelB)
    # print(delimiter)
    modelA = pd.read_csv(path_to_models + modelA_name + "_model.csv").set_index("Unnamed: 0").transpose()
    modelB = pd.read_csv(path_to_models + modelB_name + "_model.csv").set_index("Unnamed: 0").transpose()

    modelA = modelA.rename(columns=lambda x: x.replace(".", delimiter))
    modelB = modelB.rename(columns=lambda x: x.replace(".", delimiter))
    print("Begginning of matching with Coma(strategy=\"COMA_OPT\")")
    start = time.time()
    matchs_COMA_A = valentine_match(modelA, modelB, Coma(strategy="COMA_OPT"))
    print(str(len(matchs_COMA_A)) + " matches done.")
    end = time.time()
    print("Time elapsed only for matching : " + str(end - start))
    print(matchs_COMA_A)
    break

Starting matching..
FHIR_to_C-CDA_._delimiter.csv
Begginning of matching with Coma(strategy="COMA_OPT")


KeyboardInterrupt: 

In [23]:
good_match = []

ground_truth = pd.read_csv(ground_truth_path).set_index("key")
print(ground_truth)
for match in matchs_COMA_A:
    table2_key = match[0][1]
    table1_match = match[1][1]
    ground_truth_value = ground_truth.loc()[table2_key].match
    if table1_match == ground_truth_value:
        good_match.append(match)
print(str(len(good_match)) + " good matches")
print(str(len(matchs_COMA_A) - len(good_match)) + " bad matches")
print(str(len(matchs_COMA_A)) + " total matches")
print("Precision = " + str(len(good_match) / len(matchs_COMA_A)))
# break

                                                                                  match
key                                                                                    
MD_Metadata->identificationInfo->MD_DataIdentif...              contacts->address->city
MD_Metadata->identificationInfo->MD_DataIdentif...                    keywords->concept
MD_Metadata->identificationInfo->MD_DataIdentif...             temporalExtents->endDate
MD_Metadata->identificationInfo->MD_DataIdentif...                                 None
MD_Metadata->identificationInfo->MD_DataIdentif...                                 None
...                                                                                 ...
MD_Metadata->identificationInfo->MD_DataIdentif...                                 None
MD_Metadata->identificationInfo->MD_DataIdentif...  spatialExtents->area->eastLongitude
MD_Metadata->identificationInfo->MD_DataIdentif...                                 None
MD_Metadata->identificationInfo-

KeyError: 'MD_Metadata->metadataStandardName->CharacterString->XML_transform_tag_content'

In [ ]:

# def fill_excel(df, df_columns):
#     aux_row = None
#     # print(df_columns)
#     for index,row in enumerate(df.iloc):
#         if aux_row is None:
#             aux_row=df.iloc[index]
#         else:
#             for index_row,val in enumerate(row):
#                 if pd.isna(val):
#                     try:
#                         df.at[df.index[index],df_columns[index_row]] = aux_row[df_columns[index_row]]
#                     except Exception as e :
#                         print(index_row)
#                         print(df_columns)
#                         print(aux)
#                         exit(1)
#                 else:# if key != "Index"
# #
#                     break
#             aux_row=df.iloc[index]
#     return df
